In [1]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
# from pyspark.sql import functions as F

In [2]:
# Read bronze airports table
airports_df = spark.read.table("unikargo_dev.01_bronze.unikargo_airports_bronze")

In [4]:
# Add surrogate key
# sequential surrogate keys (airport_sk) ordered by IATA_CODE

# dim_airport = airports_df.withColumn("airport_sk", F.monotonically_increasing_id())

dim_airport = airports_df \
    .filter(col("iata_code").isNotNull()) \
    .dropDuplicates(["iata_code"]) \
    .withColumn("airport_sk", row_number().over(Window.orderBy("iata_code"))) \
    .select(
        "airport_sk",
        "iata_code",
        "airline",
        "city",
        "state",
        "country",
        "latitude",
        "longitude"
    )

In [7]:

# write dim_airport (dimension table)
dim_airport.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("unikargo_dev.02_silver.unikargo_dim_airport_silver")